# NLP in Action
> My journey to learn NLP

- toc: true
- branch: master
- badges: false
- comments: false
- categories: [fastpages, jupyter]

## NLP Tools

### Regular Expressions

In [17]:
import re

Match any sentence that begins with hi|hello|hey followed by space(s) and a word 

In [18]:
r = "(hi|hello|hey)[ ]*([a-z]*)"

In [19]:
print(re.match(r, 'Hello Rosa', flags=re.IGNORECASE))

<re.Match object; span=(0, 10), match='Hello Rosa'>


In [20]:
print(re.match(r, "hi ho, hi ho, it's off to work ...", flags=re.IGNORECASE))

<re.Match object; span=(0, 5), match='hi ho'>


In [21]:
print(re.match(r, "hey, what's up", flags=re.IGNORECASE))

<re.Match object; span=(0, 3), match='hey'>


Example with a complex pattern

In [22]:
r = r"[^a-z]*([y]o|[h']?ello|ok|hey|(good[ ])?(morn[gin']{0,3}|"\
     r"afternoon|even[gin']{0,3}))[\s,;:]{1,3}([a-z]{1,20})"

re_greeting = re.compile(r, flags=re.IGNORECASE)

In [23]:
print(re_greeting.match('Hello Rosa'))

<re.Match object; span=(0, 10), match='Hello Rosa'>


In [24]:
print(re_greeting.match('Hello Rosa').groups())

('Hello', None, None, 'Rosa')


In [25]:
print(re_greeting.match("Good morning Rosa"))

<re.Match object; span=(0, 17), match='Good morning Rosa'>


In [26]:
print(re_greeting.match("Good Manning Rosa"))

None


In [27]:
print(re_greeting.match('Good evening Rosa Parks').groups())

('Good evening', 'Good ', 'evening', 'Rosa')


In [28]:
print(re_greeting.match("Good Morn'n Rosa"))

<re.Match object; span=(0, 16), match="Good Morn'n Rosa">


In [29]:
print(re_greeting.match("yo Rosa"))

<re.Match object; span=(0, 7), match='yo Rosa'>


In [30]:
"""
Outputs:
match='Hello Rosa'
('Hello', None, None, 'Rosa')
match='Good morning Rosa'
null
('Good evening', 'Good ', 'evening', 'Rosa')
match="Good Morn'n Rosa"
match='yo Rosa'
"""
print('')

### Simple Chat Bot

Enter the text 'good morning rosa or hello rose'

In [33]:
my_names = set(['rosa', 'rose', 'chatty', 'chatbot', 'bot', 'chatterbot'])
curt_names = set(['hal', 'you', 'u'])
greeter_name = ''
match = re_greeting.match(input())

if match:
    at_name = match.groups()[-1]
    if at_name in curt_names:
        print("Good one.")

    elif at_name.lower() in my_names:
        print("Hi {}, How are you?".format(greeter_name))

 hello rose


Hi , How are you?


### Word Permutations

n=3 permutations with text 'Good morning Rosa!'

In [34]:
from itertools import permutations
print([" ".join(combo) for combo in permutations("Good morning Rosa!".split(), 3)])

['Good morning Rosa!', 'Good Rosa! morning', 'morning Good Rosa!', 'morning Rosa! Good', 'Rosa! Good morning', 'Rosa! morning Good']


In [35]:
"""
Output
['Good morning Rosa!',
 'Good Rosa! morning',
 'morning Good Rosa!',
 'morning Rosa! Good',
 'Rosa! Good morning',
 'Rosa! morning Good']
"""
print('')

### Count Words

Counting words using Counter. Dict output

In [36]:
from collections import Counter
print(Counter("Guten Morgen Rosa".split()))
print(Counter("Good morning morning , Rosa!".split()))

Counter({'Guten': 1, 'Morgen': 1, 'Rosa': 1})
Counter({'morning': 2, 'Good': 1, ',': 1, 'Rosa!': 1})


In [37]:
"""
Outputs:
Counter({'Guten': 1, 'Rosa': 1, 'morgen': 1})
Counter({',':1, 'Good': 1, 'Rosa!': 1, 'morning,': 2})
"""
print('')

## Tokenization